# Laboratorio 8 - Gymnasium y DQN
- Melissa Pérez Alarcón, 21385

In [1]:
import random
import collections
import math
import os
from typing import Deque, Tuple


import gymnasium as gym
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import imageio

In [ ]:
ENV_NAME = 'CartPole-v1'
SEED = 42
GAMMA = 0.99
LR = 1e-3
BATCH_SIZE = 64
BUFFER_SIZE = 10000
MIN_REPLAY_SIZE = 1000
TARGET_UPDATE_FREQ = 1000 # steps
MAX_EPISODES = 800
MAX_STEPS_PER_EPISODE = 500
EPS_START = 1.0
EPS_END = 0.01
EPS_DECAY = 50000
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

In [5]:
Transition = collections.namedtuple('Transition', ('state', 'action', 'reward', 'next_state', 'done'))


class ReplayBuffer:
    def __init__(self, capacity: int):
        self.buffer: Deque[Transition] = collections.deque(maxlen=capacity)


        def push(self, *args):
            self.buffer.append(Transition(*args))


        def sample(self, batch_size: int):
            batch = random.sample(self.buffer, batch_size)
            states = torch.tensor(np.array([t.state for t in batch]), dtype=torch.float32, device=DEVICE)
            actions = torch.tensor([t.action for t in batch], dtype=torch.int64, device=DEVICE).unsqueeze(1)
            rewards = torch.tensor([t.reward for t in batch], dtype=torch.float32, device=DEVICE).unsqueeze(1)
            next_states = torch.tensor(np.array([t.next_state for t in batch]), dtype=torch.float32, device=DEVICE)
            dones = torch.tensor([t.done for t in batch], dtype=torch.float32, device=DEVICE).unsqueeze(1)
            return states, actions, rewards, next_states, dones


        def __len__(self):
            return len(self.buffer)